### Import intel extension

In [1]:
############# code changes ###############
import intel_extension_for_pytorch as ipex
############# code changes ###############

/home/u136698/.local/lib/python3.9/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: ''If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [2]:
from  transformers  import  AutoTokenizer, AutoModelForSeq2SeqLM, pipeline

2023-10-20 09:31:49,039 - numexpr.utils - INFO - Note: detected 224 virtual cores but NumExpr set to maximum of 64, check "NUMEXPR_MAX_THREADS" environment variable.
2023-10-20 09:31:49,044 - numexpr.utils - INFO - Note: NumExpr detected 224 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
2023-10-20 09:31:49,047 - numexpr.utils - INFO - NumExpr defaulting to 8 threads.
Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


In [3]:
import torch
from tqdm.notebook import tqdm

## load model

In [4]:
model_name = "/home/u136698/Training/My_Codes/sparse-T5-QA"

In [5]:
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


### optimize the model

In [6]:
#################### code changes #################
model = model.to("xpu")
model = ipex.optimize(model, dtype=torch.float16)
#################### code changes #################

/home/u136698/.local/lib/python3.9/site-packages/intel_extension_for_pytorch/frontend.py:611: UserWarning: Conv BatchNorm folding failed during the optimize process.
  warnings.warn(
/home/u136698/.local/lib/python3.9/site-packages/intel_extension_for_pytorch/frontend.py:618: UserWarning: Linear BatchNorm folding failed during the optimize process.
  warnings.warn(


In [7]:
import pandas as pd

In [8]:
test_data = pd.read_csv("../data/test.csv")

In [9]:
test_data["input"] = test_data.apply(lambda x: f'question: {x["Question"]}  context: {x["Story"]}', axis=1)

In [10]:
test_data

,Story,Question,input
0,"CHAPTER I \n\nON THE HOUSEBOAT \n\n""Say, Tom, ...",What was Sam's mood?,question: What was Sam's mood? context: CHAPT...
1,The Uralic languages (; sometimes called Urali...,By how many?,question: By how many? context: The Uralic la...
2,The University of Wisconsin–Madison (also know...,How many schools and colleges does it have?,question: How many schools and colleges does i...
3,(CNN) -- The unexpected resignation of David P...,What happened?,question: What happened? context: (CNN) -- Th...
4,"CHAPTER THIRTY-TWO \n\nTENDER TROUBLES \n\n""Jo...",Why does Mrs. March not force her children's c...,question: Why does Mrs. March not force her ch...
...,...,...,...
28543,"CHAPTER III. \n\n""Nice customs curt'sy to grea...",Who was the Archbishop of Toledo that was of t...,question: Who was the Archbishop of Toledo tha...
28544,CHAPTER XI. \n\nIn the little dining-room of t...,Who lived there?,question: Who lived there? context: CHAPTER X...
28545,"Chapter XXX. \n\n""I shall go on through all et...",is he fair?,question: is he fair? context: Chapter XXX. \...
28546,Chapter XXXIV \n\nViolence \n\n\n\nIt had been...,Where was he to stay until this trip?,question: Where was he to stay until this trip...


### encode the input

In [11]:
encoded_inputs = tokenizer.batch_encode_plus(
    test_data["input"].to_list(),
    return_tensors='pt',
    max_length=1024,
    truncation=True,
    padding=True
)

In [12]:
size = 128

### generate the results

In [ ]:
results = []
with torch.no_grad():
    ########################### code changes ########################
    with torch.xpu.amp.autocast(enabled=True, dtype=torch.float16):
    ########################### code changes ########################
        for i in tqdm(range(encoded_inputs["input_ids"].shape[0]//size + 1)):
            payload = encoded_inputs["input_ids"][i*size:(i+1)*size].to("xpu")
            result = tokenizer.batch_decode(model.generate(payload, max_length=128), skip_special_tokens=True)
            results.extend(result)

In [ ]:
submission = pd.read_csv("./data/submission.csv")

In [54]:
submission["Answer"] = results

## simple post processing

In [55]:
submission["Answer"] = submission["Answer"].apply(lambda x: x.strip())

In [56]:
submission.to_csv("/tmp/submission.csv", index=False)

In [57]:
submission = pd.read_csv("/tmp/submission.csv")

In [58]:
submission["Answer"] = submission["Answer"].fillna("0")
submission["Answer"] = submission["Answer"].astype(str)

### save the submission file

In [59]:
submission.to_csv("../submission.csv", index=False)

In [61]:
pd.read_csv("../submission.csv").isna().sum()

Answer    0
dtype: int64